Create a 2D tensor of shape (4, 5) filled with random numbers in [-1, 1] on GPU if available, else CPU. Then: <br>
– Print its shape, mean, std, min, max.<br>
– Reshape it to (2, 10), then back to (4, 5).<br>
– Add a new dimension at axis 1 (so shape becomes (4, 1, 5)), then remove it back.<br>
– Move it to float16 if supported, else stay in float32.<br>

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"


In [3]:

inp = (torch.rand(size=(4,5))*(1+1) - 1).to(device)
print(f"Shape: {inp.shape}, mean:{torch.mean(inp):.4f}, std: {torch.std(inp):.4f}, max: {torch.max(inp)}, min: {torch.min(inp)}")
reshaped = torch.reshape(inp,(2,10))
print(f"Reshaped tensor: {reshaped}, shape : {(reshaped.shape)}")
original = torch.reshape(reshaped,(4,5))
print(f"Original tensor :{original}, shape: {original.shape}")


Shape: torch.Size([4, 5]), mean:0.2160, std: 0.5982, max: 0.9212484359741211, min: -0.8552501201629639
Reshaped tensor: tensor([[-0.0251,  0.9195,  0.1990, -0.1448,  0.8208, -0.3692, -0.2397,  0.6922,
         -0.8553,  0.8977],
        [ 0.2984,  0.8109, -0.4353,  0.4617,  0.9212, -0.7419,  0.3491,  0.3910,
          0.9089, -0.5398]], device='mps:0'), shape : torch.Size([2, 10])
Original tensor :tensor([[-0.0251,  0.9195,  0.1990, -0.1448,  0.8208],
        [-0.3692, -0.2397,  0.6922, -0.8553,  0.8977],
        [ 0.2984,  0.8109, -0.4353,  0.4617,  0.9212],
        [-0.7419,  0.3491,  0.3910,  0.9089, -0.5398]], device='mps:0'), shape: torch.Size([4, 5])


## Squeezing and unsqueezing of tensors
- manipulating a tensor's shape by adding or removing dimensions with a size of 1
- They are essential for ensuring tensors have compatible shapes for operations like broadcasting, matrix multiplication, and feeding data into neural network layers.

What they do
- torch.unsqueeze(input, dim): This function adds a new dimension with a size of 1 at the specified position (dim).
- torch.squeeze(input, dim=None): This function removes all dimensions with a size of 1. If a specific dim is provided, it removes the dimension at that position only if it has a size of 1. 


### When is unsqueeze used?
- Many neural network layers, such as convolutional layers, expect a 4D tensor with the shape (batch_size, channels, height, width). If you are processing a single image, its shape might be (channels, height, width). You can use unsqueeze(0) to add a batch dimension of size 1

In [10]:
img = torch.rand((3,228,228)).to(torch.float32)
processed_img = torch.unsqueeze(img, dim=0)

In [12]:
print(img.shape) # (C,H,W)
print(processed_img.shape) # (B,C,H,W)

torch.Size([3, 228, 228])
torch.Size([1, 3, 228, 228])


### When is Squeeze used? 

- After a computation that results in a dimension of size 1, you might need to remove it to simplify the tensor or match the expected shape for the next operation. For example, a global average pooling layer might produce an output with a dimension of size 1

In [ ]:
img2 = torch.rand((10,128,1,1))
sq_  = torch.squeeze(img2, dim=2)


In [17]:
sq_ = torch.squeeze(sq_, dim=2)

In [18]:
print(sq_.shape)
print(img2.shape)

torch.Size([10, 128])
torch.Size([10, 128, 1, 1])


In [19]:
# Explicit vs Implicit Squeezing
input = torch.rand((1,1,128,128,3))
input_ = torch.squeeze(input) # Removes all singleton dimensions by default

print(input.shape)
print(input_.shape)

torch.Size([1, 1, 128, 128, 3])
torch.Size([128, 128, 3])


- Your task is to manipulate example_tensor using squeeze and unsqueeze to achieve the target shapes and verify the results. For every step, print the shape of the resulting tensor and the tensor itself.

In [ ]:
import torch

# A complex tensor for our exercise
data = torch.arange(27).reshape(3, 3, 3).to(torch.float32)
example_tensor = data.unsqueeze(0).unsqueeze(2).unsqueeze(-1)
print(f"Initial tensor shape: {example_tensor.shape}")
print(f"Initial tensor:\n{example_tensor}")


Step 1: Universal Squeezing
- Remove all dimensions of size 1 from example_tensor without specifying a dimension.

In [22]:
squeezed_tensor = torch.squeeze(example_tensor) # implicit squeezing
print(squeezed_tensor.shape)

torch.Size([3, 3, 3])


Step 2: Selective Squeezing
- Start again with the original example_tensor. Squeeze only the batch and channel dimensions (dimensions 0 and 2) in a single operation.


In [24]:
squeezed_new = torch.squeeze(example_tensor,dim = (0,2,5))
print(squeezed_new.shape)

torch.Size([3, 3, 3])


Step 3: Sequential Unsqueezing
- Start with the torch.Size([3, 3, 3]) tensor you created in Step 1.
- Add a batch dimension at the beginning (dim=0).
- Add a channel dimension at the end (dim=-1).
- Add another dimension of size 1 between the last two dimensions (dim=3). 
- Expected final shape: torch.Size([1, 3, 3, 1, 3, 1])

In [28]:
a = torch.rand((3,3,3,))
b = torch.unsqueeze(a,dim=0)
# Since the channel dimension must be 3 and the batch dimension must be 1
c = torch.unsqueeze(b,-1).expand((1,3,3,3,3))
d = torch.unsqueeze(c,-2)
print(d.shape)

torch.Size([1, 3, 3, 3, 1, 3])


Step 4: Unsqueezing for Broadcasting
- Consider two tensors:
- A = torch.randn(3, 1, 5)
- B = torch.randn(5)
- Explain why A + B would cause an error. Then, provide the code to unsqueeze B correctly so that the addition is successful, and print the resulting shape.

In [45]:
import torch

A = torch.randn((3, 1, 5))
B = torch.randn((5))

try:
    _ = A + B
except RuntimeError as e:
    print(f"Error when adding A and B directly: {e}")

B_unsqueezed = B.unsqueeze(0).unsqueeze(0)
print(f"\nShape of A: {A.shape}")
print(f"Shape of B after unsqueezing: {B_unsqueezed.shape}")

# Perform the successful addition
C = A + B_unsqueezed
print(f"Resulting shape of C: {C.shape}")



Shape of A: torch.Size([3, 1, 5])
Shape of B after unsqueezing: torch.Size([1, 1, 5])
Resulting shape of C: torch.Size([3, 1, 5])


Step 5: Conceptual Question - The View
- Start again with the torch.Size([3, 3, 3]) tensor from Step 1 and assign it to a new variable final_tensor.
- Perform viewed_tensor = final_tensor.unsqueeze(0).
- Perform squeezed_back = viewed_tensor.squeeze(0).
- Modify an element in squeezed_back.
- Explain whether final_tensor will also be modified, and why. Provide code to demonstrate.

In [39]:
p = torch.rand((3,3,3))
q = p.unsqueeze(0)
r = q.squeeze(0)


In [41]:
r[0,0,0] = 1

In [ ]:
assert (p[0,0,0] == r[0,0,0]) # So changing